In [ ]:
!pip install mysql-connector-python langchain langchain_openai openai langchain_experimental sqlalchemy pinecone-client -q

You should consider upgrading via the '/Users/danielhernandez/.pyenv/versions/3.10.0/bin/python3.10 -m pip install --upgrade pip' command.


In [1]:
import sqlalchemy
import pandas as pd

database_username = "db"
database_password = "password"
database_ip = "ip"
database_name = "db_name"
database_port = "db_port"

database_connection = sqlalchemy.create_engine(
    "mysql+mysqlconnector://{0}:{1}@{2}:{3}/{4}".format(
        database_username, database_password, database_ip, database_port, database_name
    )
)

comments = pd.read_sql_query(
    """SELECT * FROM all_star_comment""", con=database_connection
)

comments = comments[['date', 'stars', 'comment']]
comments

,date,stars,comment
0,2021-02-27 08:29:34,5,"El servicio en este hotel es excepcional, el p..."
1,2021-02-27 08:29:34,5,"Las instalaciones son impecables, todo está mu..."
2,2021-02-27 08:29:34,4,La comida en el restaurante del hotel es delic...
3,2021-03-31 08:29:34,4,"La ubicación es perfecta, cerca de las princip..."
4,2021-03-31 08:29:34,4,"Las habitaciones son cómodas y espaciosas, ide..."
5,2021-03-31 08:29:34,5,El Wi-Fi funciona perfectamente en todo el hot...
6,2021-03-31 08:29:34,4,"El desayuno buffet es abundante y variado, con..."
7,2021-04-28 08:29:34,5,El ambiente del hotel es muy acogedor y tranqu...
8,2021-04-28 08:29:34,5,El spa y las áreas de bienestar son excelentes...
9,2021-01-31 08:29:34,4,El hotel ofrece una gran cantidad de servicios...


In [125]:
from sqlalchemy import Table, MetaData, insert, func
from sqlalchemy.orm import sessionmaker
from sqlalchemy.exc import SQLAlchemyError

def save_memory_db(user_id, company_id, messages):
    # Define metadata
    metadata = MetaData()

    # Reflect the table
    ai_memory_chats = Table('ai_memory_chats', metadata, autoload_with=database_connection)

    # Insert a new row
    with database_connection.connect() as connection:
        trans = connection.begin()
        try:
            insert_stmt = insert(ai_memory_chats).values(
                user_id=user_id,
                company_id=company_id,
                messages=messages,
                created_at=func.now(),
                updated_at=func.now()
            )
            connection.execute(insert_stmt)
            trans.commit()

            return 'Sucessfully completed'
        except SQLAlchemyError as error:
            trans.rollback()
            return error

def retrieve_memory_db(user_id):
    Session = sessionmaker(bind=database_connection)
    session = Session()
    metadata = MetaData()


    ai_memory_chats = Table('ai_memory_chats', metadata, autoload_with=database_connection)

    with database_connection.connect() as connection:
        try:
            # Create a select statement
            query = session.query(ai_memory_chats).filter(ai_memory_chats.c.user_id == user_id)

            # Execute the statement
            result = query.all()  # or use .all() to fetch all records matching the user_id

            return result[0]
            
        except SQLAlchemyError as e:
            print(f"Failed to retrieve records: {e}")

In [2]:
from getpass import getpass
import os

OPENAI_API_KEY = getpass('Enter the secret value: ')
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

### Best AI Agent (Until Now)

In [ ]:
from langchain_experimental.agents.agent_toolkits import create_python_agent
from langchain_experimental.tools.python.tool import PythonREPLTool
from langchain_openai import ChatOpenAI

def analyze_business_feedback(query):

    llm = ChatOpenAI(model='gpt-4-turbo-preview', temperature=0)

    agent_executor = create_python_agent(
        llm=llm,
        tool=PythonREPLTool(),
        verbose=True,
        agent_executor_kwargs={"handle_parsing_errors": True, "timeout": 60}
    )

    data_description = comments.to_string(index=False)

    prompt = f"""
    El objetivo de un experto analista de retroalimentación de clientes es encontrar oportunidades de mejora que vayan más allá de simplemente identificar puntos clave de buen servicio al cliente.
    Dame una respuesta corta y simple.
    Intenta responder la siguiente pregunta: {query}, analizando los siguientes datos buscando keywords de la pregunta {data_description} o dando una respuesta que pueda ser relevante.
    """

    response = agent_executor.invoke(prompt)
    return response['output']


analyze_business_feedback('Que debería mejorar de mi servicio?')


In [54]:
import gradio as gr

demo = gr.Interface(fn=analyze_business_feedback, inputs="text", outputs="text")

demo.launch(share=False)

Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.


IMPORTANT: You are using gradio version 4.28.3, however version 4.29.0 is available, please upgrade.
--------


> Entering new AgentExecutor chain...
I don't know
Observation: Invalid Format: Missing 'Action:' after 'Thought:
Thought:Soy un experto analista de retroalimentación de clientes. Intenta con una pregunta diferente, por favor.
Observation: Invalid Format: Missing 'Action:' after 'Thought:
Thought:Final Answer: Soy un experto analista de retroalimentación de clientes. Intenta con una pregunta diferente, por favor.

> Finished chain.


> Entering new AgentExecutor chain...
I don't know
Observation: Invalid Format: Missing 'Action:' after 'Thought:
Thought:Soy un experto analista de retroalimentación de clientes. Intenta con una pregunta diferente, por favor.
Observation: Invalid Format: Missing 'Action:' after 'Thought:
Thought:Final Answer: Soy un experto analista de retroalimentación de clientes. Intenta con una pregunta diferente, por favor.

> Finished chain.


> Entering n

### Running Agent with Memory

In [112]:
from langchain.memory import ConversationBufferMemory
from langchain_core.messages import SystemMessage
from langchain.chains import LLMChain
from langchain_core.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)
from langchain_openai import ChatOpenAI

data_description = comments.to_string(index=False)

# template = f"""
# El objetivo de un experto analista de retroalimentación de clientes es encontrar oportunidades de mejora que vayan más allá de simplemente identificar puntos clave de buen servicio al cliente.
# Dame una respuesta corta y simple.
# Intenta responder la siguiente pregunta: {query}, analizando los siguientes datos buscando keywords de la pregunta {data_description} o dando una respuesta que pueda ser relevante.
# """

prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content=f"""Eres un experto analista de retroalimentación de clientes es encontrar oportunidades de mejora que vayan más allá de simplemente identificar puntos clave de buen servicio al cliente.
            Tu respuesta más larga debe ser dada en dos párrafos.
            Da respuestas cortas.
            Responde analizando los siguientes datos buscando keywords de la pregunta {data_description}. """
        ),  # The persistent system prompt
        MessagesPlaceholder(
            variable_name="chat_history"
        ),  # Where the memory will be stored.
        HumanMessagePromptTemplate.from_template(
            "{human_input}"
        ),  # Where the human input will injected
    ]
)

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

### Saving memory

In [126]:
import json
from langchain.schema import messages_from_dict, messages_to_dict

extracted_messages = memory.chat_memory.messages
ingest_to_db = messages_to_dict(extracted_messages)
chat_history_json = json.dumps(ingest_to_db)
chat_history_json

save_memory_db(1, 1, 'Hola')

AttributeError: 'Insert' object has no attribute 'on_conflict_do_update'

### Retrieving memory

In [115]:
from langchain.memory.chat_message_histories.in_memory import ChatMessageHistory

def assigning_memory(user_id):
    try:
        retrieve_from_db = retrieve_memory_db(user_id)[3]
        retrieved_messages = json.loads(retrieve_from_db)
        retrieved_messages = messages_from_dict(retrieved_messages)
        retrieved_chat_history = ChatMessageHistory(messages=retrieved_messages)
        memory = ConversationBufferMemory(memory_key="chat_history", chat_memory=retrieved_chat_history, return_messages=True)
    except:
        memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)    

    return memory


In [116]:
assigning_memory(2)

ConversationBufferMemory(return_messages=True, memory_key='chat_history')

In [117]:
memory

ConversationBufferMemory(return_messages=True, memory_key='chat_history')

In [96]:
llm = ChatOpenAI(model='gpt-4-turbo-preview', temperature=0)

chat_llm_chain = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True,
    memory=memory,
)

def ask_question(input):
    output = chat_llm_chain.predict(human_input=input)
    return output

import gradio as gr

demo = gr.Interface(fn=ask_question, inputs="text", outputs="text")

demo.launch(share=False)

Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.


IMPORTANT: You are using gradio version 3.50.2, however version 4.29.0 is available, please upgrade.
--------


> Entering new LLMChain chain...
Prompt after formatting:
System: Eres un experto analista de retroalimentación de clientes es encontrar oportunidades de mejora que vayan más allá de simplemente identificar puntos clave de buen servicio al cliente.
            Tu respuesta más larga debe ser dada en dos párrafos.
            Da respuestas cortas.
            Responde analizando los siguientes datos buscando keywords de la pregunta                date  stars                                                                                                                                                                           comment
2021-02-27 08:29:34      5                                                                                   El servicio en este hotel es excepcional, el personal siempre está atento y dispuesto a ayudar.
2021-02-27 08:29:34      5                 